# PDF Chunks Visualizer

Herramienta interactiva para visualizar los rectángulos bbox de chunks sobre el PDF original.

## Características:
- **Modo Original**: Visualiza elementos directos del reconocimiento Dolphin
- **Modo Philatelic**: Visualiza chunks enriquecidos del archivo existente  
- **Modo LIVE**: Procesa datos en tiempo real con bbox corregidos

## Uso:
1. Configurar variables en la siguiente celda
2. Ejecutar todas las celdas secuencialmente
3. Usar controles interactivos para navegar

## 1. Configuración y Imports

In [1]:
# Imports necesarios
import json
import pymupdf  # PyMuPDF
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path
from collections import defaultdict

print("✓ Imports completados")

✓ Imports completados


## 2. Variables de Configuración

**Personaliza el archivo PDF a visualizar:**

In [2]:
# ========================
# CONFIGURACIÓN PRINCIPAL
# ========================

# Nombre base del archivo (sin extensión)
PDF_FILE = "OXCART22"

# Rutas automáticas (no modificar)
PDF_PATH = f"./pdfs/{PDF_FILE}.pdf"
ORIGINAL_JSON = f"./results/recognition_json/{PDF_FILE}.json"
PHILATELIC_JSON = f"./results/parsed_jsons/{PDF_FILE}_philatelic.json"

print(f"📄 Configuración para: {PDF_FILE}")
print("="*50)

# Verificar archivos disponibles
files_status = {
    "PDF": Path(PDF_PATH).exists(),
    "Original JSON": Path(ORIGINAL_JSON).exists(), 
    "Philatelic JSON": Path(PHILATELIC_JSON).exists()
}

for file_type, exists in files_status.items():
    status = "✅" if exists else "❌"
    print(f"{status} {file_type}")

print("="*50)

📄 Configuración para: OXCART22
✅ PDF
✅ Original JSON
✅ Philatelic JSON


## 3. Funciones de Utilidad

In [3]:
def load_json_data(json_path):
    """Cargar datos JSON con manejo de errores"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {json_path}")
        return None
    except json.JSONDecodeError:
        print(f"❌ Error decodificando JSON: {json_path}")
        return None

def pdf_page_to_image(pdf_path, page_num, dpi=150):
    """Convertir página PDF a imagen PIL"""
    try:
        doc = pymupdf.open(pdf_path)
        if page_num < 1 or page_num > len(doc):
            print(f"❌ Página {page_num} fuera de rango (1-{len(doc)})")
            return None, (0, 0)
        
        page = doc[page_num - 1]  # PyMuPDF usa índice base-0
        
        # Dimensiones originales  
        page_rect = page.rect
        original_width = int(page_rect.width)
        original_height = int(page_rect.height)
        
        # Renderizar con DPI específico
        mat = pymupdf.Matrix(dpi/72, dpi/72)
        pix = page.get_pixmap(matrix=mat)
        
        # Convertir a PIL Image
        img_data = pix.tobytes("png") 
        pil_image = Image.open(io.BytesIO(img_data))
        
        doc.close()
        return pil_image, (original_width, original_height)
        
    except Exception as e:
        print(f"❌ Error cargando PDF: {e}")
        return None, (0, 0)

def get_color_for_type(chunk_type):
    """Mapeo de colores para tipos de chunks"""
    colors = {
        'text': '#00FF00',      'paragraph': '#00FF00',  'title': '#00FF00',     'section': '#00FF00',
        'table': '#0000FF',     'table_row': '#4169E1', 
        'figure': '#FF0000',    'fig': '#FF0000',       'image': '#FF0000',
        'header': '#FFFF00',    'footer': '#FFFF00',
        'marginalia': '#FF8C00', 'caption': '#FF69B4',   'cap': '#FF69B4',
    }
    return colors.get(chunk_type.lower(), '#808080')

print("✅ Funciones de utilidad definidas")

✅ Funciones de utilidad definidas


In [4]:
# Necesario para pdf_page_to_image()
import io

def extract_chunks_for_page(data, page_num, data_type="original"):
    """
    Extraer chunks de una página específica según el tipo de datos.
    
    Args:
        data: Datos fuente (original o procesado)
        page_num: Número de página (1-indexado)
        data_type: 'original' para Dolphin, 'chunks' para philatelic/optimizado
        
    Returns:
        list: Lista de chunks con bbox y metadatos
    """
    chunks = []
    
    if data_type == "original":
        # Formato original Dolphin: buscar en pages->elements
        for page in data.get('pages', []):
            if page.get('page_number') == page_num:
                for elem in page.get('elements', []):
                    bbox = elem.get('bbox')
                    if bbox and len(bbox) == 4:
                        chunks.append({
                            'bbox': bbox,  # [x1, y1, x2, y2] absoluto
                            'type': elem.get('label', 'unknown'),
                            'text': elem.get('text', '')[:100] + ('...' if len(elem.get('text', '')) > 100 else ''),
                            'reading_order': elem.get('reading_order', 0),
                            'normalized': False
                        })
                break
                
    else:
        # Formato chunks: buscar en chunks->grounding  
        for chunk in data.get('chunks', []):
            grounding = chunk.get('grounding', [])
            if grounding:
                g = grounding[0]
                if g.get('page') == page_num:
                    bbox = g.get('box')
                    if bbox:  # Puede ser dict {l,t,r,b} o null
                        chunks.append({
                            'bbox': bbox,  # Coordenadas normalizadas 0-1
                            'type': chunk.get('chunk_type', 'unknown'),
                            'text': chunk.get('text', '')[:100] + ('...' if len(chunk.get('text', '')) > 100 else ''),
                            'chunk_id': chunk.get('chunk_id', ''),
                            'normalized': True
                        })
    
    return chunks

def draw_bbox_rectangles(pil_image, chunks, page_dimensions, show_labels=True, show_ids=False):
    """
    Dibujar rectángulos bbox sobre imagen PIL.
    
    Args:
        pil_image: Imagen PIL base
        chunks: Lista de chunks con bbox
        page_dimensions: (width, height) originales del PDF  
        show_labels: Mostrar tipo de chunk
        show_ids: Mostrar IDs de chunk
        
    Returns:
        PIL.Image: Imagen con rectángulos dibujados
    """
    img_with_boxes = pil_image.copy()
    draw = ImageDraw.Draw(img_with_boxes)
    
    original_width, original_height = page_dimensions
    img_width, img_height = pil_image.size
    
    # Factores de escala PDF -> imagen renderizada
    scale_x = img_width / original_width if original_width > 0 else 1
    scale_y = img_height / original_height if original_height > 0 else 1
    
    for i, chunk in enumerate(chunks):
        bbox = chunk['bbox']
        color = get_color_for_type(chunk['type'])
        
        if chunk.get('normalized', False):
            # Coordenadas normalizadas 0-1 -> píxeles imagen
            x1 = int(bbox['l'] * img_width)
            y1 = int(bbox['t'] * img_height) 
            x2 = int(bbox['r'] * img_width)
            y2 = int(bbox['b'] * img_height)
        else:
            # Coordenadas absolutas PDF -> píxeles imagen
            x1 = int(bbox[0] * scale_x)
            y1 = int(bbox[1] * scale_y)
            x2 = int(bbox[2] * scale_x) 
            y2 = int(bbox[3] * scale_y)
        
        # Dibujar rectángulo
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # Etiquetas opcionales
        if show_labels or show_ids:
            label_parts = []
            if show_ids and 'chunk_id' in chunk:
                label_parts.append(f"ID:{chunk['chunk_id'].split(':')[-1]}")
            if show_labels:
                label_parts.append(chunk['type'])
            
            if label_parts:
                label = " | ".join(label_parts)
                try:
                    bbox_text = draw.textbbox((x1, y1-20), label)
                    draw.rectangle([bbox_text[0]-2, bbox_text[1]-2, bbox_text[2]+2, bbox_text[3]+2], 
                                 fill='white', outline=color)
                    draw.text((x1, y1-20), label, fill='black')
                except:
                    pass  # Skip si error con textbbox
    
    return img_with_boxes

print("✅ Funciones de extracción y renderizado definidas")

✅ Funciones de extracción y renderizado definidas


## 4. Carga de Datos y Análisis Básico

In [5]:
# ====================================
# CARGAR Y ANALIZAR DATOS DISPONIBLES  
# ====================================

print("🔄 Cargando datos...")

# Cargar archivos JSON
original_data = load_json_data(ORIGINAL_JSON) if Path(ORIGINAL_JSON).exists() else None
philatelic_data = load_json_data(PHILATELIC_JSON) if Path(PHILATELIC_JSON).exists() else None

# Análisis de datos originales
if original_data:
    total_pages = len(original_data.get('pages', []))
    total_elements = sum(len(page.get('elements', [])) for page in original_data.get('pages', []))
    print(f"✅ Original: {total_pages} páginas, {total_elements} elementos")
    
    # Muestra por página
    for page in original_data['pages'][:3]:
        page_num = page.get('page_number')
        elements = len(page.get('elements', []))
        print(f"   📄 Página {page_num}: {elements} elementos")
    
    if total_pages > 3:
        print(f"   📄 ... y {total_pages - 3} páginas más")
else:
    print("❌ No se encontraron datos originales")
    total_pages = 1

# Análisis de datos philatelic  
if philatelic_data:
    total_chunks = len(philatelic_data.get('chunks', []))
    meta = philatelic_data.get('extraction_metadata', {})
    version = meta.get('enrichment_version', 'N/A')
    print(f"✅ Philatelic: {total_chunks} chunks, versión {version}")
else:
    print("❌ No se encontraron datos philatelic")

# Configurar rango de páginas
MAX_PAGES = total_pages if original_data else 1
print(f"\n📊 Páginas disponibles: 1 a {MAX_PAGES}")
print("="*50)

🔄 Cargando datos...
✅ Original: 21 páginas, 274 elementos
   📄 Página 1: 23 elementos
   📄 Página 2: 14 elementos
   📄 Página 3: 12 elementos
   📄 ... y 18 páginas más
✅ Philatelic: 235 chunks, versión philately-advanced-v3.0

📊 Páginas disponibles: 1 a 21


## 5. Procesamiento LIVE para Validación Bbox

**Generar datos philatelic con bbox corregidos en tiempo real:**

In [6]:
def create_live_philatelic_data():
    """
    Procesa datos originales aplicando transformación completa con bbox corregidos.
    
    Este proceso:
    1. Extrae páginas del PDF como imágenes PNG  
    2. Crea page_dims_provider funcional
    3. Aplica transformación dolphin_to_oxcart con dimensiones correctas
    4. Enriquece con metadatos philatelic
    5. Valida que los bbox no sean null
    
    MEJORAS IMPLEMENTADAS:
    - Parámetros optimizados para chunks más largos (mejor alineamiento con modelo ideal)
    - para_max_chars: 1500 (aumentado de 1000)
    - target_avg_length: 300 (aumentado de 150)
    - Agrupación más agresiva para reducir fragmentación
    
    Returns:
        dict: Datos OXCART con bbox válidos o None si falla
    """
    if not original_data:
        print("❌ No hay datos originales disponibles")
        return None
    
    print("🔄 Iniciando procesamiento LIVE MEJORADO...")
    print("="*40)
    
    try:
        # Importar y recargar módulos para última versión
        from dolphin_transformer import transform_dolphin_to_oxcart_preserving_labels
        from philatelic_patterns import enrich_all_chunks_advanced_philatelic
        from PIL import Image
        import importlib, os, pymupdf
        
        import dolphin_transformer, philatelic_patterns
        importlib.reload(dolphin_transformer)
        importlib.reload(philatelic_patterns)
        
        from dolphin_transformer import transform_dolphin_to_oxcart_preserving_labels
        from philatelic_patterns import enrich_all_chunks_advanced_philatelic
        
        # Paso 1: Crear directorio y extraer páginas PDF
        pages_dir = "./results/pages"
        os.makedirs(pages_dir, exist_ok=True)
        print(f"📁 Directorio creado: {pages_dir}")
        
        print("📄 Extrayendo páginas PDF...")
        doc = pymupdf.open(PDF_PATH)
        total_pages = len(doc)
        
        for page_num in range(total_pages):
            page = doc[page_num]
            pix = page.get_pixmap()
            output_path = f"{pages_dir}/page_{page_num+1:03d}.png"
            pix.save(output_path)
            
            if page_num < 3:  # Mostrar progreso primeras 3
                print(f"   ✅ Página {page_num+1}: {pix.width}x{pix.height}px")
        
        doc.close()
        print(f"✅ Extraídas {total_pages} páginas como PNG")
        
        # Paso 2: Definir page_dims_provider
        def get_page_dimensions(page_num):
            img_path = f"./results/pages/page_{page_num:03d}.png"
            try:
                if os.path.exists(img_path):
                    return Image.open(img_path).size
                return None, None
            except Exception as e:
                print(f"   ❌ Error página {page_num}: {e}")
                return None, None
        
        # Paso 3: Test dimensiones
        test_dims = get_page_dimensions(1)
        print(f"🔍 Test página 1: {test_dims}")
        
        # Paso 4: Aplicar transformación con parámetros MEJORADOS
        print("⚙️  Aplicando transformación MEJORADA...")
        recognition_results = original_data['pages']
        
        live_data = transform_dolphin_to_oxcart_preserving_labels(
            recognition_results,
            doc_id=PDF_FILE,
            page_dims_provider=get_page_dimensions,
            para_max_chars=1500,  # ← MEJORADO: Aumentado de 1000
            target_avg_length=300,  # ← MEJORADO: Aumentado de 150  
            max_chunk_length=1200,  # ← MEJORADO: Aumentado de 800
            table_row_block_size=None,
            optimize_for_rag=True
        )
        print("✅ Transformación base completada")
        
        # Paso 5: Validar bbox generados
        valid_bbox = sum_null_bbox = 0
        sample_boxes = []
        
        for chunk in live_data.get('chunks', []):
            grounding = chunk.get('grounding', [])
            if grounding:
                bbox = grounding[0].get('box')
                if bbox and bbox != None:
                    valid_bbox += 1
                    if len(sample_boxes) < 3:
                        page_num = grounding[0].get('page')
                        sample_boxes.append(f"📄 P{page_num}: {bbox}")
                else:
                    sum_null_bbox += 1
        
        total_chunks = len(live_data.get('chunks', []))
        
        # Calcular estadísticas de longitud
        text_lengths = [len(chunk.get('text', '')) for chunk in live_data.get('chunks', [])]
        avg_length = sum(text_lengths) / len(text_lengths) if text_lengths else 0
        
        print("="*40)
        print("📊 RESULTADOS DE VALIDACIÓN:")
        print(f"   ✅ Chunks con bbox válidos: {valid_bbox}")  
        print(f"   ❌ Chunks con bbox null: {sum_null_bbox}")
        print(f"   📊 Total chunks: {total_chunks}")
        print(f"   📏 Longitud promedio: {avg_length:.1f} chars")
        print(f"   📐 Rango longitud: {min(text_lengths) if text_lengths else 0} - {max(text_lengths) if text_lengths else 0} chars")
        
        if sample_boxes:
            print("\n📝 Ejemplos de bbox válidos:")
            for sample in sample_boxes:
                print(f"   {sample}")
        
        # Paso 6: Enriquecer con metadatos philatelic si bbox OK
        if valid_bbox > 0:
            print("\n✅ ¡Bbox funcionando correctamente!")
            print("🔄 Aplicando enriquecimiento philatelic...")
            live_data = enrich_all_chunks_advanced_philatelic(live_data)
            print("✅ Enriquecimiento completado")
            
            # Estadísticas finales
            final_chunks = len(live_data.get('chunks', []))
            final_lengths = [len(chunk.get('text', '')) for chunk in live_data.get('chunks', [])]
            final_avg = sum(final_lengths) / len(final_lengths) if final_lengths else 0
            
            print(f"\n📈 ESTADÍSTICAS FINALES:")
            print(f"   📦 Chunks finales: {final_chunks}")
            print(f"   📏 Longitud promedio final: {final_avg:.1f} chars")
            print(f"   🎯 Mejora vs ideal (352.5 chars): {(final_avg/352.5)*100:.1f}%")
        else:
            print("\n❌ Los bbox siguen null - problema persistente")
            
        print("="*40)
        return live_data
        
    except Exception as e:
        print(f"❌ Error en procesamiento LIVE: {e}")
        import traceback
        traceback.print_exc()
        return None

# Ejecutar procesamiento LIVE automáticamente
print("🚀 Ejecutando procesamiento LIVE MEJORADO...")
live_philatelic_data = create_live_philatelic_data()

🚀 Ejecutando procesamiento LIVE MEJORADO...
🔄 Iniciando procesamiento LIVE MEJORADO...
📁 Directorio creado: ./results/pages
📄 Extrayendo páginas PDF...
   ✅ Página 1: 612x792px
   ✅ Página 2: 612x792px
   ✅ Página 3: 612x792px
✅ Extraídas 21 páginas como PNG
🔍 Test página 1: (612, 792)
⚙️  Aplicando transformación MEJORADA...
Info: Table row segmentation disabled, keeping table as single chunk on page 7
Info: Table row segmentation disabled, keeping table as single chunk on page 7
Info: Table row segmentation disabled, keeping table as single chunk on page 8
Info: Table row segmentation disabled, keeping table as single chunk on page 8
Info: Table row segmentation disabled, keeping table as single chunk on page 9
Info: Table row segmentation disabled, keeping table as single chunk on page 11
Info: Table row segmentation disabled, keeping table as single chunk on page 11
Info: Table row segmentation disabled, keeping table as single chunk on page 12
Info: Table row segmentation disabled

## 6. Configuración de Interfaz Interactiva

**Configurar widgets y controles de navegación:**

In [7]:
# ===============================
# CREAR WIDGETS Y CONTROLES
# ===============================

# Determinar modos disponibles
available_modes = []
if original_data:
    available_modes.append(("📄 Original (Dolphin)", "original"))
if philatelic_data:
    available_modes.append(("💎 Philatelic (Archivo)", "philatelic")) 
if original_data:  # Agregar LIVE si tenemos datos originales
    available_modes.append(("⭕ LIVE Philatelic", "live_philatelic"))  # Cambié emoji

if not available_modes:
    print("❌ No hay datos disponibles para visualizar")
    widgets_created = False
else:
    print(f"✅ Modos disponibles: {len(available_modes)}")
    for name, _ in available_modes:
        print(f"   {name}")
    
    # Widget selector de modo
    mode_dropdown = widgets.Dropdown(
        options=available_modes,
        value=available_modes[0][1],
        description='Modo:',
        style={'description_width': '80px'},
        layout={'width': '200px'}
    )
    
    # Widget selector de página  
    page_slider = widgets.IntSlider(
        value=1,
        min=1,
        max=MAX_PAGES,
        step=1,
        description='Página:',
        style={'description_width': '80px'},
        layout={'width': '300px'}
    )
    
    # Checkboxes de opciones
    show_labels_check = widgets.Checkbox(
        value=True,
        description='Mostrar etiquetas',
        style={'description_width': '120px'}
    )
    
    show_ids_check = widgets.Checkbox(
        value=False,
        description='Mostrar IDs',
        style={'description_width': '120px'}
    )
    
    # Botones de navegación
    prev_button = widgets.Button(
        description="◀ Anterior",
        button_style='info',
        layout={'width': '100px'}
    )
    
    next_button = widgets.Button(
        description="Siguiente ▶", 
        button_style='info',
        layout={'width': '100px'}
    )
    
    # Variable global para tracking de la función de actualización
    _global_update_function = None
    
    # Eventos de botones - se conectarán después
    def navigate_prev(b):
        if page_slider.value > 1:
            page_slider.value -= 1
            # La actualización se dispara automáticamente por el observer del slider
    
    def navigate_next(b):
        if page_slider.value < MAX_PAGES:
            page_slider.value += 1
            # La actualización se dispara automáticamente por el observer del slider
    
    prev_button.on_click(navigate_prev)
    next_button.on_click(navigate_next)
    
    # Organizar controles en layout limpio
    controls = widgets.VBox([
        widgets.HTML("<h4>🎛️ Controles de Visualización</h4>"),
        widgets.HBox([mode_dropdown, page_slider]),
        widgets.HBox([prev_button, next_button]),
        widgets.HBox([show_labels_check, show_ids_check]),
        widgets.HTML("<hr>")
    ])
    
    widgets_created = True
    print("✅ Widgets de interfaz creados")

print("="*50)

✅ Modos disponibles: 3
   📄 Original (Dolphin)
   💎 Philatelic (Archivo)
   ⭕ LIVE Philatelic
✅ Widgets de interfaz creados


## 7. Función Principal de Visualización

**Lógica central para renderizar páginas con bbox:**

In [8]:
def visualize_current_page():
    """
    Función principal de visualización que renderiza la página actual 
    con rectángulos bbox según el modo seleccionado.
    
    Modos soportados:
    - 'original': Elementos directos de Dolphin recognition
    - 'philatelic': Chunks del archivo philatelic existente  
    - 'live_philatelic': Chunks procesados en tiempo real con bbox corregidos
    """
    # Obtener parámetros actuales de widgets
    current_mode = mode_dropdown.value
    current_page = page_slider.value
    show_labels = show_labels_check.value
    show_ids = show_ids_check.value
    
    print(f"📖 VISUALIZANDO PÁGINA {current_page}")
    print(f"🔧 Modo: {current_mode}")
    print("="*50)
    
    # Cargar imagen base del PDF
    pil_image, page_dims = pdf_page_to_image(PDF_PATH, current_page)
    if pil_image is None:
        print("❌ Error cargando página del PDF")
        return
    
    print(f"✅ Página cargada: {pil_image.size[0]}x{pil_image.size[1]}px")
    
    # Seleccionar fuente de datos según modo
    if current_mode == "original":
        data, data_type = original_data, "original"
    elif current_mode == "live_philatelic":
        if live_philatelic_data is None:
            print("❌ Datos LIVE no disponibles")
            print("   Ejecuta primero las celdas de procesamiento LIVE")
            return
        data, data_type = live_philatelic_data, "chunks"
        print("🔴 Usando datos LIVE con bbox corregidos")
    else:  # philatelic archivo
        data, data_type = philatelic_data, "chunks"
    
    # Extraer chunks para esta página específica
    chunks = extract_chunks_for_page(data, current_page, data_type)
    
    if not chunks:
        print(f"⚠️  Sin chunks en página {current_page}")
        plt.figure(figsize=(12, 16))
        plt.imshow(pil_image)
        plt.axis('off')
        plt.title(f"Página {current_page} - Sin chunks detectados")
        plt.show()
        return
    
    print(f"📊 Chunks encontrados: {len(chunks)}")
    
    # Renderizar imagen con bbox
    img_with_boxes = draw_bbox_rectangles(
        pil_image, chunks, page_dims, show_labels, show_ids
    )
    
    # Mostrar resultado
    plt.figure(figsize=(14, 18))
    plt.imshow(img_with_boxes)
    plt.axis('off')
    
    # Títulos sin emojis para evitar warnings de matplotlib
    mode_name = {
        "original": "Original Dolphin",
        "philatelic": "Philatelic Archivo", 
        "live_philatelic": "LIVE Philatelic"
    }.get(current_mode, current_mode)
    
    plt.title(f"Página {current_page} - {mode_name} ({len(chunks)} chunks)")
    plt.tight_layout()
    plt.show()
    
    # === ESTADÍSTICAS DETALLADAS ===
    print("\\n📊 ESTADÍSTICAS:")
    print(f"   📄 Página: {current_page}")
    print(f"   📦 Total chunks: {len(chunks)}")
    
    # Validación bbox para modo LIVE
    if current_mode == "live_philatelic":
        valid_bbox = sum(1 for c in chunks if c.get('bbox') and c['bbox'] != None)
        print(f"   ✅ Bbox válidos: {valid_bbox}/{len(chunks)}")
    
    # Conteo por tipos
    type_counts = defaultdict(int)
    for chunk in chunks:
        type_counts[chunk['type']] += 1
    
    print("\\n🏷️  TIPOS DETECTADOS:")
    for chunk_type, count in sorted(type_counts.items()):
        print(f"   🟦 {chunk_type}: {count}")
    
    # Lista detallada (primeros 8)
    print("\\n📝 DETALLE DE CHUNKS (primeros 8):")
    for i, chunk in enumerate(chunks[:8]):
        chunk_id = chunk.get('chunk_id', f"#{i+1}")
        text = chunk['text'][:50] + "..." if len(chunk['text']) > 50 else chunk['text']
        bbox_ok = "✅" if chunk.get('bbox') and chunk['bbox'] != None else "❌"
        
        print(f"   {i+1:2d}. [{chunk['type']}] {bbox_ok} {text}")
        if 'chunk_id' in chunk:
            print(f"       ID: {chunk_id}")
    
    if len(chunks) > 8:
        print(f"   ... y {len(chunks) - 8} chunks más")
    
    print("="*50)

print("✅ Función de visualización definida")

✅ Función de visualización definida


## 8. Interfaz Interactiva Activa

**¡Usa los controles para navegar y explorar!**

In [9]:
# INTERFAZ COMPLETA AUTO-CONTENIDA (SOLUCIÓN DEFINITIVA)

print("🚀 CREANDO INTERFAZ COMPLETA")
print("="*50)

# === PASO 1: CREAR WIDGETS DESDE CERO ===
# Determinar modos disponibles
available_modes = []
if original_data:
    available_modes.append(("📄 Original (Dolphin)", "original"))
if philatelic_data:
    available_modes.append(("💎 Philatelic (Archivo)", "philatelic"))
if live_philatelic_data:
    available_modes.append(("🔴 LIVE Philatelic", "live_philatelic"))

if not available_modes:
    print("❌ No hay datos disponibles")
else:
    print(f"✅ Modos: {[name for name, _ in available_modes]}")
    
    # Crear widgets nuevos
    mode_dropdown_new = widgets.Dropdown(
        options=available_modes,
        value=available_modes[0][1],
        description='Modo:',
        style={'description_width': '80px'},
        layout={'width': '220px'}
    )
    
    page_slider_new = widgets.IntSlider(
        value=1,
        min=1,
        max=MAX_PAGES,
        step=1,
        description='Página:',
        style={'description_width': '80px'},
        layout={'width': '300px'}
    )
    
    show_labels_new = widgets.Checkbox(
        value=True,
        description='Etiquetas',
        style={'description_width': '80px'}
    )
    
    show_ids_new = widgets.Checkbox(
        value=False,
        description='IDs',
        style={'description_width': '40px'}
    )
    
    # Botones navegación  
    prev_btn = widgets.Button(description="◀ Ant", button_style='info', layout={'width': '80px'})
    next_btn = widgets.Button(description="Sig ▶", button_style='info', layout={'width': '80px'})
    
    # === PASO 2: CREAR FUNCIÓN DE VISUALIZACIÓN AUTOCONTENIDA ===
    def render_page():
        """Función interna que renderiza con widgets locales"""
        current_mode = mode_dropdown_new.value
        current_page = page_slider_new.value
        show_labels = show_labels_new.value
        show_ids = show_ids_new.value
        
        print(f"📖 PÁGINA {current_page} | MODO: {current_mode}")
        print("="*50)
        
        # Cargar imagen
        pil_image, page_dims = pdf_page_to_image(PDF_PATH, current_page)
        if pil_image is None:
            print("❌ Error cargando página")
            return
        
        # Seleccionar datos según modo
        if current_mode == "original":
            data, data_type = original_data, "original"
        elif current_mode == "live_philatelic":
            data, data_type = live_philatelic_data, "chunks"
        else:  # philatelic
            data, data_type = philatelic_data, "chunks"
        
        # Extraer chunks
        chunks = extract_chunks_for_page(data, current_page, data_type)
        
        if not chunks:
            print(f"⚠️  Sin chunks en página {current_page}")
            plt.figure(figsize=(12, 16))
            plt.imshow(pil_image)
            plt.axis('off')
            plt.title(f"Página {current_page} - Sin chunks")
            plt.show()
            return
        
        print(f"📊 Chunks: {len(chunks)}")
        
        # Renderizar
        img_with_boxes = draw_bbox_rectangles(pil_image, chunks, page_dims, show_labels, show_ids)
        
        plt.figure(figsize=(14, 18))
        plt.imshow(img_with_boxes)
        plt.axis('off')
        
        mode_names = {"original": "Original", "philatelic": "Philatelic", "live_philatelic": "LIVE"}
        mode_name = mode_names.get(current_mode, current_mode)
        plt.title(f"Página {current_page} - {mode_name} ({len(chunks)} chunks)")
        plt.tight_layout()
        plt.show()
        
        # Estadísticas básicas
        type_counts = defaultdict(int)
        for chunk in chunks:
            type_counts[chunk['type']] += 1
        
        print("🏷️  TIPOS:")
        for chunk_type, count in sorted(type_counts.items()):
            print(f"   • {chunk_type}: {count}")
        print("="*50)
    
    # === PASO 3: CREAR OUTPUT Y FUNCIÓN DE ACTUALIZACIÓN ===
    output_area = widgets.Output()
    
    def update_display(*args):
        """Actualizar display cuando cambie cualquier widget"""
        output_area.clear_output(wait=True)
        with output_area:
            render_page()
    
    # === PASO 4: CONECTAR EVENTOS ===
    # Observers para cambios automáticos
    mode_dropdown_new.observe(update_display, 'value')
    page_slider_new.observe(update_display, 'value')
    show_labels_new.observe(update_display, 'value')
    show_ids_new.observe(update_display, 'value')
    
    # Botones de navegación
    def go_prev(b):
        if page_slider_new.value > 1:
            page_slider_new.value -= 1
    
    def go_next(b):
        if page_slider_new.value < MAX_PAGES:
            page_slider_new.value += 1
    
    prev_btn.on_click(go_prev)
    next_btn.on_click(go_next)
    
    # === PASO 5: LAYOUT Y DISPLAY ===
    controls_layout = widgets.VBox([
        widgets.HTML("<h4>🎛️ PDF Chunks Visualizer</h4>"),
        widgets.HBox([mode_dropdown_new, page_slider_new]),
        widgets.HBox([prev_btn, next_btn, show_labels_new, show_ids_new]),
        widgets.HTML("<hr>")
    ])
    
    display(controls_layout)
    display(output_area)
    
    # === PASO 6: RENDERIZADO INICIAL ===
    update_display()
    
    print("✅ INTERFAZ ACTIVA")
    print("📌 Cambia el dropdown para alternar entre modos")
    print("📌 Usa el slider o botones para navegar páginas")

🚀 CREANDO INTERFAZ COMPLETA
✅ Modos: ['📄 Original (Dolphin)', '💎 Philatelic (Archivo)', '🔴 LIVE Philatelic']


Output()

✅ INTERFAZ ACTIVA
📌 Cambia el dropdown para alternar entre modos
📌 Usa el slider o botones para navegar páginas


In [10]:
# ⚠️ CELDA REMOVIDA - USAR SOLO LA CELDA ANTERIOR
# Esta celda causaba conflictos con múltiples outputs.
# La celda anterior (17) contiene la interfaz corregida.

print("⚠️ Esta celda ha sido deshabilitada.")
print("📌 Usar la celda anterior (Interfaz Interactiva Simplificada)")
print("✅ Si los controles no funcionan, reinicia el kernel y ejecuta todo de nuevo")

⚠️ Esta celda ha sido deshabilitada.
📌 Usar la celda anterior (Interfaz Interactiva Simplificada)
✅ Si los controles no funcionan, reinicia el kernel y ejecuta todo de nuevo


### 8.1 Reiniciar Interfaz (ejecutar solo si hay problemas)

**Si ves repeticiones, ejecuta esta celda para limpiar todo:**

In [11]:
print("🔧 INFORMACIÓN DEL SISTEMA")
print("="*50)
print(f"📄 Archivo PDF: {PDF_FILE}")
print(f"📊 Páginas totales: {MAX_PAGES}")

if original_data:
    total_elements = sum(len(page.get('elements', [])) for page in original_data.get('pages', []))
    print(f"📦 Elementos originales (Dolphin): {total_elements}")

if philatelic_data:
    total_chunks = len(philatelic_data.get('chunks', []))
    meta = philatelic_data.get('extraction_metadata', {})
    version = meta.get('enrichment_version', 'N/A')
    print(f"💎 Chunks philatelic: {total_chunks}")
    print(f"🏷️  Versión enriquecimiento: {version}")

if live_philatelic_data:
    live_chunks = len(live_philatelic_data.get('chunks', []))
    print(f"🔴 Chunks LIVE procesados: {live_chunks}")

print(f"\n🎨 LEYENDA DE COLORES:")
print("="*30)
color_legend = {
    '🟢 Verde': 'Texto (párrafos, títulos, secciones)',
    '🔵 Azul': 'Tablas y filas de tabla', 
    '🔴 Rojo': 'Figuras, imágenes y elementos visuales',
    '🟡 Amarillo': 'Headers y footers',
    '🟠 Naranja': 'Marginalia y notas',
    '🌸 Rosa': 'Captions y leyendas'
}

for color, description in color_legend.items():
    print(f"   {color}: {description}")

print(f"\n💡 CONSEJOS DE USO:")
print("="*20)
print("• Usa 📄 Original para ver elementos detectados por Dolphin")
print("• Usa 💎 Philatelic para chunks del archivo existente") 
print("• Usa 🔴 LIVE para validar corrección de bbox en tiempo real")
print("• Navega con botones ◀▶ o arrastra el slider")
print("• Activa/desactiva etiquetas e IDs según necesidad")
print("="*50)

🔧 INFORMACIÓN DEL SISTEMA
📄 Archivo PDF: OXCART22
📊 Páginas totales: 21
📦 Elementos originales (Dolphin): 274
💎 Chunks philatelic: 235
🏷️  Versión enriquecimiento: philately-advanced-v3.0
🔴 Chunks LIVE procesados: 166

🎨 LEYENDA DE COLORES:
   🟢 Verde: Texto (párrafos, títulos, secciones)
   🔵 Azul: Tablas y filas de tabla
   🔴 Rojo: Figuras, imágenes y elementos visuales
   🟡 Amarillo: Headers y footers
   🟠 Naranja: Marginalia y notas
   🌸 Rosa: Captions y leyendas

💡 CONSEJOS DE USO:
• Usa 📄 Original para ver elementos detectados por Dolphin
• Usa 💎 Philatelic para chunks del archivo existente
• Usa 🔴 LIVE para validar corrección de bbox en tiempo real
• Navega con botones ◀▶ o arrastra el slider
• Activa/desactiva etiquetas e IDs según necesidad


## 9. Información del Sistema

**Resumen técnico y leyenda de colores:**

In [ ]:
# from philatelic_patterns import  *
# save_json(live_philatelic_data, f"./results/parsed_jsons/{PDF_FILE}_philatelic.json")

'./results/parsed_jsons/OXCART22_philatelic.json'

In [ ]:
# live_philatelic_data

{'doc_id': 'OXCART22',
 'source': 'dolphin',
 'page_count': 21,
 'extraction_date': '2025-08-22T15:45:03.402051Z',
 'metadata': {'title': '', 'author': '', 'language': 'und'},
 'chunks': [{'chunk_id': 'OXCART22:001:3-3:0',
   'chunk_type': 'text',
   'text': 'Published quarterly by the\nsociety of Costa Rica collectors',
   'grounding': [{'page': 1,
     'box': {'l': 0.401961, 't': 0.214646, 'r': 0.759804, 'b': 0.265152}}],
   'metadata': {'labels': ['title'],
    'reading_order_range': [3, 3],
    'part_index': None,
    'quality_score': 0.5,
    'combined_chunks': 1,
    'entities': {},
    'topics': {'secondary': [], 'tags': [], 'confidence': 0.5}}},
  {'chunk_id': 'OXCART22:001:4-4:0',
   'chunk_type': 'header',
   'text': 'EDITOR',
   'grounding': [{'page': 1,
     'box': {'l': 0.132353, 't': 0.292929, 'r': 0.204248, 'b': 0.316919}}],
   'metadata': {'labels': ['sec'],
    'reading_order_range': [4, 4],
    'part_index': None,
    'quality_score': 0.5,
    'combined_chunks': 1,
  